In [8]:
import os
whereami = os.environ.get("WHEREAMI")
if whereami=="shane_local":
    print(whereami)
elif whereami=="databricks":
    print(whereami)
    import dbutils
    dbutils.library.restartPython()

In [5]:
%pip install -q torch 
%pip install -q transformers 
%pip install -q accelerate 
%pip install -q bitsandbytes 
%pip install -q langchain 
%pip install -q sentence-transformers 
%pip install -q faiss-gpu 
%pip install -q openpyxl 
%pip install -q pacmap

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.
